In [1]:
from pybacktestchain import data_module
from pybacktestchain.data_module import * 
from datetime import date, timedelta, datetime


from my_package_project.data_treatment import *
from my_package_project.graphs  import *
from my_package_project.operations import *



In [2]:
tickers = ['SPY', 'TLT', 'GLD', "AAPL", "MSFT"]  # S&P 500 Index, 20+ Year Treasury Bond, Gold (commodities)
data = get_stocks_data(tickers=tickers, start_date='2000-01-01', end_date='2020-12-31').dropna()
# data['Date'] = data['Date'].astype(str).str[:10] #we had an error with non-UTC dates
# data['Date'] = pd.to_datetime(data['Date'])
data

,Date,Open,High,Low,Close,Adj Close,Volume,ticker
0,2000-01-03 00:00:00-05:00,148.250000,148.250000,143.875000,145.437500,92.692924,8164300,SPY
1,2000-01-04 00:00:00-05:00,143.531250,144.062500,139.640625,139.750000,89.068016,8089800,SPY
2,2000-01-05 00:00:00-05:00,139.937500,141.531250,137.250000,140.000000,89.227409,12177900,SPY
3,2000-01-06 00:00:00-05:00,139.625000,141.500000,137.750000,137.750000,87.793388,6227200,SPY
4,2000-01-07 00:00:00-05:00,140.312500,145.750000,140.062500,145.750000,92.892136,8066500,SPY
...,...,...,...,...,...,...,...,...
5278,2020-12-23 00:00:00-05:00,223.110001,223.559998,220.800003,221.020004,214.069565,18699600,MSFT
5279,2020-12-24 00:00:00-05:00,221.419998,223.610001,221.199997,222.750000,215.745163,10550600,MSFT
5280,2020-12-28 00:00:00-05:00,224.449997,226.029999,223.020004,224.960007,217.885666,17933500,MSFT
5281,2020-12-29 00:00:00-05:00,226.309998,227.179993,223.580002,224.149994,217.101166,17403200,MSFT


In [3]:
data_module = DataModule(data=data)
info = Information(s = timedelta(data_module.data.size/len(tickers)), data_module=data_module)
t = datetime(2020,12,29)
moments = FirstTwoMoments.compute_information(info, t=t)


In [4]:
riskparityportfolio =  data_treatment.RiskParity.compute_portfolio_riskparity(info, t=t, information_set=moments)
riskparityportfolio

{'AAPL': 0.24345221693620614,
 'GLD': 0.21834842514184602,
 'MSFT': 0.13558971251775756,
 'SPY': 0.09307758066345044,
 'TLT': 0.30953206474073985}

In [5]:
vol = data_treatment.portfolio_volatility(riskparityportfolio,information_set=moments)
vol

28.874189288453984

In [6]:
data_treatment.RiskParity.compute_portfolio_riskparity_voltarget_leverage(info, t=t, information_set=moments, target_volatility=vol, leverage_factor=1)

INFO:root:Original Volatility: 28.874189288453984


{'AAPL': 0.24345221693620614,
 'GLD': 0.21834842514184602,
 'MSFT': 0.13558971251775756,
 'SPY': 0.09307758066345044,
 'TLT': 0.30953206474073985}

In [7]:
visualizer = PortfolioVisualizer(portfolio=riskparityportfolio, information_set=moments)
visualizer.plot_portfolio_weights()


In [8]:
data_treatment.compute_risk_contributions(riskparityportfolio, moments)

{'AAPL': 0.2, 'GLD': 0.2, 'MSFT': 0.2, 'SPY': 0.2, 'TLT': 0.2}

In [9]:
visualizer.plot_risk_allocation_pie()

In [12]:
test = Backtest_up(
    initial_date = datetime(2013, 1, 1),
    final_date = datetime(2023, 12, 31),
    information_class = FirstTwoMoments,
    risk_model=StopLoss,
)

In [13]:
test.run_backtest()

INFO:root:Running backtest from 2013-01-01 00:00:00 to 2023-12-31 00:00:00.
INFO:root:Retrieving price data for universe
/Users/marc/Desktop/anaconda3/envs/pybacktestchain/lib/python3.11/site-packages/pandas/core/frame.py:11211: RuntimeWarning:

Degrees of freedom <= 0 for slice

/Users/marc/Desktop/anaconda3/envs/pybacktestchain/lib/python3.11/site-packages/numpy/lib/function_base.py:2748: RuntimeWarning:

divide by zero encountered in divide

/Users/marc/Desktop/anaconda3/envs/pybacktestchain/lib/python3.11/site-packages/numpy/lib/function_base.py:2748: RuntimeWarning:

invalid value encountered in multiply

INFO:root:Stop loss triggered for GLD at 2013-01-03 00:00:00. Selling all shares.
INFO:root:Stop loss triggered for SPY at 2013-01-03 00:00:00. Selling all shares.
INFO:root:Stop loss triggered for TLT at 2013-01-29 00:00:00. Selling all shares.
INFO:root:-----------------------------------
INFO:root:Rebalancing portfolio at 2013-01-31 00:00:00
INFO:root:-------------------------

INFO:root:Backtest completed. Final portfolio value: 1080920.4183273315
